In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches


# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# # list_values.append('coarse')

# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
strategy_version_input=705
attribute_id_input=5030
formatted_attribute = 'Tensile Strength'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

16932


16932

In [8]:
df['buckets'].explode().value_counts()[0:50] 

Fixings & Fasteners                  16929
Fastener Extractors & Applicators        3
Name: buckets, dtype: int64

In [9]:
df_fasteners=df[df['buckets'].astype(str)=='Fixings & Fasteners']
print(len(df_fasteners))
# df_fasteners

16929


In [18]:
regex_pattern_tensile=r'(?i)(tensile)|()'
df_fasteners['matches_tensile']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern_tensile, x))

regex_pattern=r'(?i)(\d\d?\d?,\d\d\d.?P.?S.?I.?)|(P.?S.?I.?.?\d\d?\d?,\d\d\d)|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_tensile'].astype(str)!='[]')])) 
tensile_psi=df_fasteners[(df_fasteners['matches'].astype(str)!='[]')]
len(tensile_psi)

3174


4179

In [19]:
tensile_psi['matches']=tensile_psi['matches'].apply(lambda x: re.sub(r'P.S.I.','PSI',str(x)))
tensile_psi['matches'].explode().value_counts()

['150,000 PSI']                   1347
['180,000 PSI']                    532
['145,000 PSI']                    417
['150,800 PSI']                    310
['170,000 PSI']                    251
['60,000 PSI']                     246
['200,000 PSI']                    220
['176,100 PSI']                    177
['176,946 PSI']                    126
['150,000 PSI', '60,000 PSI']      106
['140,000 PSI']                     72
['176,000 PSI']                     71
['142,100 PSI']                     58
['6,000 PSI']                       36
['100,000 PSI']                     35
['135,000 PSI']                     33
['175,000 PSI']                     26
['27,000 PSI']                      26
['150,000 PSI ']                    22
['12,000 PSI']                      19
['74,000 PSI']                      18
['60,000 PSI', '75,000 PSI']        11
['95,000 PSI']                      11
['145,000 PSI', '135,000 PSI']       9
Name: matches, dtype: int64

In [32]:
tensile_psi[tensile_psi['matches'].astype(str)=="['145,000 PSI', '135,000 PSI']"].head()

product_id external_id                                product_name  \
587   1173656428      305710   7/16-14 x 1-1/ Flat Socket Head Cap Screw   
1350  1173656429      305712  7/16-14 x 1-1/2 Flat Socket Head Cap Screw   
1879  1173656427      305708     7/16-14 x 1" Flat Socket Head Cap Screw   
4574  1173656435      305724     7/16-14 x 3" Flat Socket Head Cap Screw   
4575  1173656434      305720  7/16-14 x 2-1/2 Flat Socket Head Cap Screw   

                                                                                                                                                                                                                                     long_desc  \
587   Alloy steel - Rockwell C39-C44 up to 1/2 inch diameter. Rockwell C37-C44 over 1/2 inch diameter. Conforms to ANSI B18.3. Black with oil dipped finish. Tensile Strength: 145,000 P.S.I. up through 1/2”, 135,000 P.S.I. 5/8” and larger.   
1350  Alloy steel - Rockwell C39-C44 up to 1/2 inch diameter. Rockwell C37-C44 over 1/2 inch diameter. Conforms to ANSI B18.3. Black with oil dipped finish. Tensile Strength: 145,000 P.S.I. up through 1/2”, 135,000 P.S.I. 5/8” and larger.   
1879  Alloy steel - Rockwell C39-C44 up to 1/2 inch diameter. Rockwell C37-C44 over 1/2 inch diameter. Conforms to ANSI B18.3. Black with oil dipped finish. Tensile Strength: 145,000 P.S.I. up through 1/2”, 135,000 P.S.I. 5/8” and larger.   
4574  Alloy steel - Rockwell C39-C44 up to 1/2 inch diameter. Rockwell C37-C44 over 1/2 inch diameter. Conforms to ANSI B18.3. Black with oil dipped finish. Tensile Strength: 145,000 P.S.I. up through 1/2”, 135,000 P.S.I. 5/8” and larger.   
4575  Alloy steel - Rockwell C39-C44 up to 1/2 inch diameter. Rockwell C37-C44 over 1/2 inch diameter. Conforms to ANSI B18.3. Black with oil dipped finish. Tensile Strength: 145,000 P.S.I. up through 1/2”, 135,000 P.S.I. 5/8” and larger.   

      customer_id attributes              buckets  bucket_id values  \
587           137       None  Fixings & Fasteners       7265   None   
1350          137       None  Fixings & Fasteners       7265   None   
1879          137       None  Fixings & Fasteners       7265   None   
4574          137       None  Fixings & Fasteners       7265   None   
4575          137       None  Fixings & Fasteners       7265   None   

     matches_tensile                         matches  
587        [Tensile]  ['145,000 PSI', '135,000 PSI']  
1350       [Tensile]  ['145,000 PSI', '135,000 PSI']  
1879       [Tensile]  ['145,000 PSI', '135,000 PSI']  
4574       [Tensile]  ['145,000 PSI', '135,000 PSI']  
4575       [Tensile]  ['145,000 PSI', '135,000 PSI']

In [31]:
tensile_psi[tensile_psi['matches'].astype(str)=="['150,000 PSI', '60,000 PSI']"].head()

product_id external_id                       product_name  \
38   1173660531      375908  1/2 Alloy Steel Split Lock Washer   
74   1173660088      376104        1/4 Alloy Steel Flat Washer   
78   1173660014      376108        1/2 Alloy Steel Flat Washer   
217  1173648565     3820506   5/16-18 x 3/4 Hex Head Cap Screw   
689  1173648605     3820606    3/8-16 x 3/4 Hex Head Cap Screw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
38   ARMOR COAT™ FINISH - Exceptionally corrosion resistant. Ideal for the most demanding, harsh environments where conventional fasteners quickly fail. PASSES 1000 HOUR SALT SPRAY - At 1000 hours Armor Coat™ is still providing complete protection while zinc plating, zinc yellow dichromate and even hot dip galvanizing have long since failed. 10 TIMES MORE CORROSION RESISTANT - In standardized testing, only stainless steel held up as well as Armor Coat™. UP TO 250% STRONGER THAN STAINLESS STEEL - Armor Coat™ Grade 8 provides 150,000 P.S.I. minimum tensile strength versus 60,000 P.S.I. and up for 18-8 and 316 stainless steel.   
74   ARMOR COAT™ FINISH - Exceptionally corrosion resistant. Ideal for the most demanding, harsh environments where conventional fasteners quickly fail. PASSES 1000 HOUR SALT SPRAY - At 1000 hours Armor Coat™ is still providing complete protection while zinc plating, zinc yellow dichromate and even hot dip galvanizing have long since failed. 10 TIMES MORE CORROSION RESISTANT - In standardized testing, only stainless steel held up as well as Armor Coat™*. UP TO 250% STRONGER THAN STAINLESS STEEL - Armor Coat Grade 8 provides 150,000 P.S.I. minimum tensile strength versus 60,000 P.S.I. and up for 18-8 and 316 stainless steel.   
78   ARMOR COAT™ FINISH - Exceptionally corrosion resistant. Ideal for the most demanding, harsh environments where conventional fasteners quickly fail. PASSES 1000 HOUR SALT SPRAY - At 1000 hours Armor Coat™ is still providing complete protection while zinc plating, zinc yellow dichromate and even hot dip galvanizing have long since failed. 10 TIMES MORE CORROSION RESISTANT - In standardized testing, only stainless steel held up as well as Armor Coat™*. UP TO 250% STRONGER THAN STAINLESS STEEL - Armor Coat Grade 8 provides 150,000 P.S.I. minimum tensile strength versus 60,000 P.S.I. and up for 18-8 and 316 stainless steel.   
217   ARMOR COAT™ FINISH - Exceptionally corrosion resistant. Ideal for the most demanding, harsh environments where conventional fasteners quickly fail. PASSES 1000 HOUR SALT SPRAY - At 1000 hours Armor Coat™ is still providing complete protection while zinc plating, zinc yellow dichromate and even hot dip galvanizing have long since failed. 10 TIMES MORE CORROSION RESISTANT - In standardized testing, only stainless steel held up as well as Armor Coat™. UP TO 250% STRONGER THAN STAINLESS STEEL - Armor Coat Grade 8 provides 150,000 P.S.I. minimum tensile strength versus 60,000 P.S.I. and up for 18-8 and 316 stainless steel.   
689   ARMOR COAT™ FINISH - Exceptionally corrosion resistant. Ideal for the most demanding, harsh environments where conventional fasteners quickly fail. PASSES 1000 HOUR SALT SPRAY - At 1000 hours Armor Coat™ is still providing complete protection while zinc plating, zinc yellow dichromate and even hot dip galvanizing have long since failed. 10 TIMES MORE CORROSION RESISTANT - In standardized testing, only stainless steel held u

In [24]:
regex_pattern_psi=r'(?i)(p.?s.?i.?)|()'
df_fasteners['matches_psi']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern_psi, x))

regex_pattern=r'(?i)(\d\d?\d?,\d\d\d.?P.?S.?I.?)|(P.?S.?I.?.?\d\d?\d?,\d\d\d)|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_tensile'].astype(str)=='[]')])) 
psi=df_fasteners[(df_fasteners['matches'].astype(str)=='[]')&(df_fasteners['matches_psi'].astype(str)!='[]')]
print(len(psi))
psi[0:500]

1005
836


product_id external_id  \
37    1173654821      446808   
66    1173659947      373804   
79    1173659960      373808   
80    1173659949      373906   
81    1173659943      373810   
83    1173659563       85654   
159   1173654814      447212   
216   1173659973     3738051   
236   1173647684      261754   
239   1173647685      261755   
241   1173647683      261753   
286   1173659353       85101   
532   1173653075       65420   
549   1173659569       85646   
553   1173659966      373816   
558   1173659333       85198   
625   1173651861      332702   
627   1173651852      465912   
645   1173651786      465820   
646   1173651797      465505   
647   1173651616      348608   
676   1173651159      345907   
682   1173651652      345710   
698   1173651519      344818   
699   1173651513      344906   
700   1173651524      344909   
704   1173651459      349906   
707   1173651341      349804   
714   1173651274      345020   
715   1173651230      345106   
797   1173651158      345908   
824   1173653060       65407   
826   1173659482       85671   
830   1173651734      349406   
831   1173651709      467189   
834   1173651500      344816   
836   1173651493      344808   
838   1173651558      344924   
843   1173651533      344918   
844   1173651499      344814   
845   1173651504      344912   
847   1173651510      344914   
848   1173651492      344810   
849   1173651479      349907   
851   1173651147      345808   
852   1173651215      349812   
853   1173651187     3450101   
872   1173651122      343824   
873   1173651157      345809   
874   1173651151      345804   
922   1173660048     3739052   
923   1173659961      373807   
939   1173651849      466012   
941   1173638129      404431   
948   1173659370       85113   
954   1173659977     3738141   
955   1173659492       85679   
967   1173660038     3738091   
968   1173659964      373904   
969   1173659944      373805   
973   1173659338       85557   
978   1173659312       85115   
1006  1173651776      465510   
1016  1173651779      465806   
1110  1173660051     3738052   
1111  1173659564       85650   
1143  1173651144      345914   
1310  1173652239       62510   
1436  1173659361       85112   
1444  1173651823      349710   
1445  1173651815      348524   
1458  1173659549       85655   
1467  1173652234       62511   
1474  1173659556       85669   
1479  1173660061     3739071   
1598  1173659559       85682   
1612  1173659519       85657   
1614  1173659558       85681   
1615  1173659560       85642   
1616  1173659522       85676   
1620  1173659528       85678   
1703  1173651731      348508   
1712  1173651603      348604   
1762  1173659562       85661   
1764  1173659552       85665   
1765  1173659566       85649   
1766  1173659536       85662   
1767  1173659537       85677   
1768  1173659553       85675   
1769  1173659547       85658   
1770  1173659567       85641   
1771  1173659565       85660   
1772  1173659557       85653   
1773  1173659496       85652   
1774  1173659539       85643   
1775  1173659498       85667   
1776  1173659355       85559   
1781  1173659331       85560   
1782  1173659352       85110   
1783  1173659327       85117   
1784  1173659316       85107   
1785  1173659351       85119   
1786  1173659344       85561   
1787  1173659328       85191   
1790  1173659356       85116   
2012  1173659525       85666   
2016  1173659367       85562   
2134  1173646707      483260   
2210  1173656313      348200   
2211  1173656311      348201   
2213  1173651227      345009   
2248  1173659930      373806   
2280  1173654805      446912   
2337  1173674039  C3109X12X4   
2338  1173674038  C3109X16X6   
2339  1173674030   C3109X4X2   
2340  1173674020   C3109X8X2   
2341  1173674018  C3109X12X6   
2342  1173674007  C3109X3220   
2343  1173674000  C3109X3224   
2344  1173673990  C3109X16X8   
2345  1173673988  C3109X2016   
2346  1173673984   C3109X6X4   
2347  1173673977  C3109X1612   
2348  

In [26]:
regex_pattern_dash=r'(?i)(\d?\d?\d?\,?\d\d?\d?\-\d\d?\d?,\d\d\d.?P.?S.?I.?)|(P.?S.?I.?.?\d?\d?\d?\,?\d\d?\d?\-\d\d?\d?,\d\d\d)|()'
df_fasteners['matches_dash']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern_dash, x))
print(len(df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_dash'].astype(str)!='[]')])) 
df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_dash'].astype(str)!='[]')]

45


product_id external_id  \
612    1173650859     3312406   
634    1173650882     3312410   
712    1173650851     3312820   
880    1173650875     3312508   
881    1173650868     3312620   
882    1173650869     3312606   
1969   1173650862     3312412   
5587   1173650886     3312516   
5588   1173650884     3312510   
5589   1173650883     3312420   
5590   1173650880     3312416   
5591   1173650876     3312520   
5592   1173650864     3312610   
5593   1173650857     3312812   
5594   1173650853     3312612   
7186   1173650866     3312404   
7432   1173650855     3312808   
8033   1173650852     3312608   
8038   1173650850     3312816   
8250   1173650889     3312604   
8252   1173650887     3312524   
8255   1173650885     3312512   
8256   1173650879     3312414   
8257   1173650878     3312504   
8258   1173650877     3312614   
8259   1173650872     3312806   
8267   1173650871     3312506   
8268   1173650870     3312624   
8272   1173650861     3312810   
8273   1173650858     3312814   
8274   1173650856     3312424   
9307   1173650854     3312408   
11586  1173650873     3312514   
11596  1173650863     3312824   
13387  1173652147      413563   
13394  1173652146      413567   
13396  1173652145      413564   
13397  1173652144      413574   
13398  1173652143      413565   
13399  1173652142      413568   
13406  1173652141      413570   
13410  1173652139      413572   
13416  1173652138      413566   
15195  1173650865     3312616   
15751  1173652137      413576   

                                            product_name  \
612       1/4-20 x 3/4 Silicon Bronze Hex Head Cap Screw   
634     1/4-20 x 1-1/4 Silicon Bronze Hex Head Cap Screw   
712     1/2-13 x 2-1/2 Silicon Bronze Hex Head Cap Screw   
880       5/16-18 x 1" Silicon Bronze Hex Head Cap Screw   
881     3/8-16 x 2-1/2 Silicon Bronze Hex Head Cap Screw   
882       3/8-16 x 3/4 Silicon Bronze Hex Head Cap Screw   
1969    1/4-20 x 1-1/2 Silicon Bronze Hex Head Cap Screw   
5587      5/16-18 x 2" Silicon Bronze Hex Head Cap Screw   
5588   5/16-18 x 1-1/4 Silicon Bronze Hex Head Cap Screw   
5589    1/4-20 x 2-1/2 Silicon Bronze Hex Head Cap Screw   
5590       1/4-20 x 2" Silicon Bronze Hex Head Cap Screw   
5591   5/16-18 x 2-1/2 Silicon Bronze Hex Head Cap Screw   
5592    3/8-16 x 1-1/4 Silicon Bronze Hex Head Cap Screw   
5593    1/2-13 x 1-1/2 Silicon Bronze Hex Head Cap Screw   
5594    3/8-16 x 1-1/2 Silicon Bronze Hex Head Cap Screw   
7186      1/4-20 x 1/2 Silicon Bronze Hex Head Cap Screw   
7432       1/2-13 x 1" Silicon Bronze Hex Head Cap Screw   
8033       3/8-16 x 1" Silicon Bronze Hex Head Cap Screw   
8038       1/2-13 x 2" Silicon Bronze Hex Head Cap Screw   
8250      3/8-16 x 1/2 Silicon Bronze Hex Head Cap Screw   
8252      5/16-18 x 3" Silicon Bronze Hex Head Cap Screw   
8255   5/16-18 x 1-1/2 Silicon Bronze Hex Head Cap Screw   
8256    1/4-20 x 1-3/4 Silicon Bronze Hex Head Cap Screw   
8257     5/16-18 x 1/2 Silicon Bronze Hex Head Cap Screw   
8258    3/8-16 x 1-3/4 Silicon Bronze Hex Head Cap Screw   
8259      1/2-13 x 3/4 Silicon Bronze Hex Head Cap Screw   
8267     5/16-18 x 3/4 Silicon Bronze Hex Head Cap Screw   
8268       3/8-16 x 3" Silicon Bronze Hex Head Cap Screw   
8272    1/2-13 x 1-1/4 Silicon Bronze Hex Head Cap Screw   
8273    1/2-13 x 1-3/4 Silicon Bronze Hex Head Cap Screw   
8274       1/4-20 x 3" Silicon Bronze Hex Head Cap Screw   
9307       1/4-20 x 1" Silicon Bronze Hex Head Cap Screw   
11586  5/16-18 x 1-3/4 Silicon Bronze Hex Head Cap Screw   
11596      1/2-13 x 3" Silicon Bronze Hex Head Cap Screw   
13387                                3/16 Drill Rod (W1)   
13394                                7/16 Drill Rod (W1)   
13396                                 1/4 Drill Rod (W1)   
13397                                 7/8 Drill Rod (W1)   
13398                                5/16 Drill Rod (W1)   
13399                                 1/2 Drill Rod (W1)   
13406                       

In [27]:
# 180,000 TO 200,000 P.S.I.

regex_pattern_to=r'(?i)(\d?\d?\d?\,?\d\d?\d?\s?To\s?\d\d?\d?,\d\d\d.?P.?S.?I.?)|(P.?S.?I.?.?\d?\d?\d?\,?\d\d?\d?\s?To\s?\d\d?\d?,\d\d\d)|()'
df_fasteners['matches_to']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern_to, x))
print(len(df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_to'].astype(str)!='[]')])) 
df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_to'].astype(str)!='[]')]

220


product_id external_id                               product_name  \
287    1173649994      981012          5/8-11 x 1-1/2 Hex Head Cap Screw   
289    1173650084      980928         9/16-12 x 3-1/2 Hex Head Cap Screw   
308    1173650128      981620            1"-8 x 2-1/2 Hex Head Cap Screw   
309    1173650083      980636          3/8-16 x 4-1/2 Hex Head Cap Screw   
310    1173650078      980728         7/16-14 x 3-1/2 Hex Head Cap Screw   
311    1173650125      990514         5/16-24 x 1-3/4 Hex Head Cap Screw   
312    1173650135      990720         7/16-20 x 2-1/2 Hex Head Cap Screw   
313    1173649997      981448              7/8-9 x 6" Hex Head Cap Screw   
314    1173650110      990610          3/8-24 x 1-1/4 Hex Head Cap Screw   
315    1173650119      990708            7/16-20 x 1" Hex Head Cap Screw   
316    1173650115      990412          1/4-28 x 1-1/2 Hex Head Cap Screw   
317    1173650117      991008             5/8-18 x 1" Hex Head Cap Screw   
318    1173650073      980618          3/8-16 x 2-1/4 Hex Head Cap Screw   
319    1173650053      980924            9/16-12 x 3" Hex Head Cap Screw   
320    1173650045      981644            1"-8 x 5-1/2 Hex Head Cap Screw   
321    1173650102      990406            1/4-28 x 3/4 Hex Head Cap Screw   
322    1173650038      981624               1"-8 x 3" Hex Head Cap Screw   
323    1173650101      990716            7/16-20 x 2" Hex Head Cap Screw   
324    1173650097      991216             3/4-16 x 2" Hex Head Cap Screw   
325    1173650034      981014          5/8-11 x 1-3/4 Hex Head Cap Screw   
326    1173650029      980912         9/16-12 x 1-1/2 Hex Head Cap Screw   
327    1173650089      981008             5/8-11 x 1" Hex Head Cap Screw   
328    1173650076      990616             3/8-24 x 2" Hex Head Cap Screw   
329    1173650074      991048             5/8-18 x 6" Hex Head Cap Screw   
330    1173650070      990812          1/2-20 x 1-1/2 Hex Head Cap Screw   
331    1173650057      990916            9/16-18 x 2" Hex Head Cap Screw   
332    1173650047      980748            7/16-14 x 6" Hex Head Cap Screw   
333    1173650044      990820          1/2-20 x 2-1/2 Hex Head Cap Screw   
334    1173650056      990848             1/2-20 x 6" Hex Head Cap Screw   
335    1173650068      980732            7/16-14 x 4" Hex Head Cap Screw   
336    1173650065      980916            9/16-12 x 2" Hex Head Cap Screw   
337    1173650052      981218          3/4-10 x 2-1/4 Hex Head Cap Screw   
338    1173650062      980532            5/16-18 x 4" Hex Head Cap Screw   
339    1173650058      980420          1/4-20 x 2-1/2 Hex Head Cap Screw   
340    1173650049      990840             1/2-20 x 5" Hex Head Cap Screw   
341    1173650043      991240             3/4-16 x 5" Hex Head Cap Screw   
342    1173650036      981212          3/4-10 x 1-1/2 Hex Head Cap Screw   
343    1173650027      980716            7/16-14 x 2" Hex Head Cap Screw   
344    1173650010      980840             1/2-13 x 5" Hex Head Cap Screw   
345    1173650013      991028          5/8-18 x 3-1/2 Hex Head Cap Screw   
346    1173650007      991228          3/4-16 x 3-1/2 Hex Head Cap Screw   
347    1173649995      980516            5/16-18 x 2" Hex Head Cap Screw   
348    1173649967      980624             3/8-16 x 3" Hex Head Cap Screw   
349    1173649961      981032             5/8-11 x 4" Hex Head Cap Screw   
350    1173649970      981432              7/8-9 x 4" Hex Head Cap Screw   
351    1173649979      980824             1/2-13 x 3" Hex Head Cap Screw   
352    1173649937      981264             3/4-10 x 8" Hex Head Cap Screw   
353    1173649958      980816             1/2-13 x 2" Hex Head Cap Screw   
355    1173649931      980864             1/2-13 x 8" Hex Head Cap Screw   
356    1173649936      981256             3/4-10 x 7" Hex Head Cap Screw   
357    1173649939      981280            3/4-10 x 10" Hex Head Cap Screw   
800    1173649980      980814          1/2-13 x 1-3/4 H

In [28]:
regex_pattern_yield_strength=r'(?i)(yield\s?strength)|()'
df_fasteners['matches_yield']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern_yield_strength, x))
print(len(df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_yield'].astype(str)!='[]')])) 
df_fasteners[(df_fasteners['matches'].astype(str)!='[]')&(df_fasteners['matches_yield'].astype(str)!='[]')]

11


product_id external_id         product_name  \
13355  1173652158      413542   1/8 Unthreaded Rod   
13360  1173652157      413543  3/16 Unthreaded Rod   
13367  1173652156      413554   7/8 Unthreaded Rod   
13371  1173652154      413547  7/16 Unthreaded Rod   
13372  1173652153      413544   1/4 Unthreaded Rod   
13373  1173652152      413556    1" Unthreaded Rod   
13378  1173652151      413546   3/8 Unthreaded Rod   
13380  1173652150      413552   3/4 Unthreaded Rod   
13381  1173652149      413550   5/8 Unthreaded Rod   
13385  1173652148      413548   1/2 Unthreaded Rod   
16377  1173652155      413545  5/16 Unthreaded Rod   

                                                                                                                                                                                                                                                                                                          long_desc  \
13355  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13360  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13367  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13371  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13372  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13373  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13378  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13380  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13381  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locking pins, push rod shafts, and adjustable rods. Zinc plated.   
13385  Specially annealed for forming, threading, and machining. Easily brazed, welded or forged. Made of low carbon steel. Approximately 75,000 P.S.I. tensile, 60,000 P.S.I. yield strength. Used for shafts, axles, linkage rods, anchor bolts, locki

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [85]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- not_tube- upload_Shawn.csv',index=False)

In [86]:
looks_good('Bed Bath & Beyond', attribute, df, not_tube)